In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 55.2 MB/s eta 0:00:00


In [7]:
!pip show fsspec
!pip show gcsfs
!pip show datasets


Name: fsspec
Version: 2024.9.0
Summary: File-system specification
Home-page: https://github.com/fsspec/filesystem_spec
Author: 
Author-email: 
License: BSD 3-Clause License

Copyright (c) 2018, Martin Durant
All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

* Redistributions of source code must retain the above copyright notice, this
  list of conditions and the following disclaimer.

* Redistributions in binary form must reproduce the above copyright notice,
  this list of conditions and the following disclaimer in the documentation
  and/or other materials provided with the distribution.

* Neither the name of the copyright holder nor the names of its
  contributors may be used to endorse or promote products derived from
  this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
AND ANY EXPRESS

In [1]:
!pip install datasets==2.14.4  fsspec==2024.9.0




In [4]:
import json

# Open and read the JSON file
with open('/content/Ecommerce_FAQ_Chatbot_dataset.json', 'r') as file:
    data = json.load(file)  # Parse the JSON data into a Python object



In [5]:
data['questions'][1]

{'question': 'What payment methods do you accept?',
 'answer': 'We accept major credit cards, debit cards, and PayPal as payment methods for online orders.'}

In [6]:
# Extract the list of questions from the 'questions' key in the data
questions = data['questions']

# Now you can extract questions and answers
faq_questions = [faq['question'] for faq in questions]
faq_answers = [faq['answer'] for faq in questions]


In [7]:
faq_questions[:5]

['How can I create an account?',
 'What payment methods do you accept?',
 'How can I track my order?',
 'What is your return policy?',
 'Can I cancel my order?']

In [8]:
faq_answers[:5]

["To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process.",
 'We accept major credit cards, debit cards, and PayPal as payment methods for online orders.',
 "You can track your order by logging into your account and navigating to the 'Order History' section. There, you will find the tracking information for your shipment.",
 'Our return policy allows you to return products within 30 days of purchase for a full refund, provided they are in their original condition and packaging. Please refer to our Returns page for detailed instructions.',
 'You can cancel your order if it has not been shipped yet. Please contact our customer support team with your order details, and we will assist you with the cancellation process.']

In [9]:
from sentence_transformers import SentenceTransformer

# Initialize the SentenceTransformer model
model =  SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [12]:
from sentence_transformers import losses, InputExample, SentenceTransformer
from torch.utils.data import DataLoader
from datasets import Dataset

# Modified Fit Method:
train_loss = losses.CosineSimilarityLoss(model=model)

# Prepare InputExamples for training
train_examples = []
for q, a in zip(faq_questions, faq_answers):
    train_examples.append(InputExample(texts=[q, a], label=1.0))

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

num_epochs = 10
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1)

model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=warmup_steps,
          show_progress_bar=True)


/usr/local/lib/python3.11/dist-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


Step,Training Loss


In [28]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
model_save_path = '/content/drive/My Drive/mymodel'

In [31]:
model.save(model_save_path)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [32]:
model = SentenceTransformer(model_save_path)

In [39]:
answer_embeddings = model.encode(faq_answers)
question_embeddings = model.encode(faq_questions)

In [40]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search_faiss_top_k(query, k=5):
    # Encode the query to obtain its embedding
    query_embedding = model.encode(query, convert_to_numpy=True).reshape(1, -1).astype('float32')

    # Perform search in the FAISS index
    distances, indices = index.search(query_embedding, k)

    # Calculate cosine similarity for the top-k results
    top_k_results = []
    for i in range(k):
        answer_index = indices[0][i]
        answer_embedding = answer_embeddings[answer_index].reshape(1, -1)

        # Compute cosine similarity
        similarity = cosine_similarity(query_embedding, answer_embedding)[0][0]

        top_k_results.append({
            "answer": faq_answers[answer_index],
            "cosine_similarity": similarity,
        })

    return top_k_results


In [41]:
query = "How do I create an account?"
top_k = 3  # Number of results to retrieve

# Search for top-k results
results = search_faiss_top_k(query, k=top_k)

# Output the results
for rank, result in enumerate(results, start=1):
    print(f"Rank {rank}:")
    print(f"Answer: {result['answer']}")
    print(f"Cosine Similarity: {result['cosine_similarity']:.4f}")
    print()


Rank 1:
Answer: To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process.
Cosine Similarity: 0.9330

Rank 2:
Answer: We accept major credit cards, debit cards, and PayPal as payment methods for online orders.
Cosine Similarity: 0.4248

Rank 3:
Answer: Yes, you can return a product purchased during a promotional event. The refund will be processed based on the amount paid after any applicable discounts.
Cosine Similarity: 0.2346

